In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 20:19:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [3]:
df_green.registerTempTable('green')

/Users/jafa/opt/anaconda3/lib/python3.9/site-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [6]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [7]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [8]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [9]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [10]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [20]:
df_yellow_revenue.show(10)


+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2021-01-27 17:00:00|  70|             153.2|             3|
|2020-02-09 06:00:00| 229|            717.36|            38|
|2021-06-06 11:00:00|  13|            588.95|            28|
|2020-02-18 12:00:00| 259|             53.21|             2|
|2020-10-04 12:00:00| 264| 359.3400000000001|            25|
|2020-08-03 05:00:00| 151|             89.21|             6|
|2020-03-08 16:00:00|  50|3427.0499999999993|           194|
|2021-01-18 17:00:00|  79| 826.4999999999998|            58|
|2021-06-20 02:00:00|   7|53.760000000000005|             4|
|2020-08-22 17:00:00| 168|151.45000000000002|             4|
+-------------------+----+------------------+--------------+
only showing top 10 rows



### Joining two large tables
##### Spark can join two tables quite easily. The syntax is easy to understand.

In [21]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

# Materialized the result
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

df_join.show(5)

+-------------------+----+-----------------+--------------------+------------------+---------------------+
|               hour|zone|     green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+-----------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   4|             null|                null|1004.3000000000002|                   57|
|2020-01-01 00:00:00|  14|             null|                null|               8.8|                    1|
|2020-01-01 00:00:00|  24|             87.6|                   3| 754.9500000000002|                   45|
|2020-01-01 00:00:00|  25|            531.0|                  26|            324.35|                   16|
|2020-01-01 00:00:00|  33|317.2700000000001|                  11|            255.56|                    8|
+-------------------+----+-----------------+--------------------+------------------+---------------------+
only showing top 5 rows



In [22]:
# Read the result previously created.
df_join = spark.read.parquet('data/report/revenue/total')

In [23]:
df_join.printSchema()

root
 |-- hour: timestamp (nullable = true)
 |-- zone: integer (nullable = true)
 |-- green_amount: double (nullable = true)
 |-- green_number_records: long (nullable = true)
 |-- yellow_amount: double (nullable = true)
 |-- yellow_number_records: long (nullable = true)



In [24]:
df_join.show(5)

+-------------------+----+------------------+--------------------+-----------------+---------------------+
|               hour|zone|      green_amount|green_number_records|    yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+-----------------+---------------------+
|2020-01-01 00:00:00|  15|              null|                null|            34.09|                    1|
|2020-01-01 00:00:00|  29|              61.3|                   1|             null|                 null|
|2020-01-01 00:00:00|  40|168.98000000000002|                   8|            89.97|                    5|
|2020-01-01 00:00:00|  42| 799.7599999999999|                  52|635.3500000000001|                   46|
|2020-01-01 00:00:00|  45|              null|                null|732.4800000000001|                   42|
+-------------------+----+------------------+--------------------+-----------------+---------------------+
only showing top 5 rows



## Read the zones.



In [25]:
!wget https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv

--2023-03-09 20:26:27--  https://s3.amazonaws.com/nyc-tlc/misc/taxi+_zone_lookup.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.249.254, 52.217.14.62, 52.216.61.72, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.249.254|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12322 (12K) [application/octet-stream]
Saving to: 'taxi+_zone_lookup.csv.1'

taxi+_zone_lookup.c 100%[===================>]  12.03K  --.-KB/s    in 0.001s  

2023-03-09 20:26:29 (9.90 MB/s) - 'taxi+_zone_lookup.csv.1' saved [12322/12322]



In [26]:
df = spark.read.option("header", "true").csv('taxi+_zone_lookup.csv')

In [29]:
# df.write.parquet('zones') // already exists
df_zones = spark.read.parquet('zones/')
df_zones.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [30]:
df_zones.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [32]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [34]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')

###### What exactly Spark is doing
Each executor processes a partition of Revenue DataFrame.
Zones DataFrame is a small table.
Because Zones is very small, each executor gets a copy of the entire Zones DataFrame and merges it with their partition of Revenue DataFrame within memory.
Spark broadcast joins are perfect for joining a large DataFrame with a small DataFrame.
Spark can "broadcast" a small DataFrame by sending all the data in that small DataFrame to all nodes in the cluster.
After the small DataFrame is broadcasted, Spark can perform a join without shuffling any of the data in the large DataFrame.
This is really (really!) fast.